In [1]:
import requests
import json
import os

# load the environment variables
from dotenv import load_dotenv
load_dotenv()

# Bring your packages onto the path
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# import functions from src/mlb_scores.py
from mlb_scores import get_mlb_scores, get_mlb_odds, get_correct_upsets, create_tweet

In [2]:
# get yesterday's date as string
import datetime
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')

# date = '2022-08-14'
date = yesterday

In [3]:
score_match_dict = get_mlb_scores(date)
score_match_dict

Connected to MLB API


{73663: {'home_team': 'Guardians',
  'away_team': 'Tigers',
  'home_score': 4,
  'away_score': 1,
  'winner': 'Home'},
 74002: {'home_team': 'Guardians',
  'away_team': 'Tigers',
  'home_score': 5,
  'away_score': 7,
  'winner': 'Away'},
 74005: {'home_team': 'Reds',
  'away_team': 'Phillies',
  'home_score': 3,
  'away_score': 4,
  'winner': 'Away'},
 74006: {'home_team': 'Marlins',
  'away_team': 'Padres',
  'home_score': 3,
  'away_score': 0,
  'winner': 'Home'},
 73999: {'home_team': 'Nationals',
  'away_team': 'Cubs',
  'home_score': 5,
  'away_score': 4,
  'winner': 'Home'},
 74000: {'home_team': 'Yankees',
  'away_team': 'Rays',
  'home_score': 0,
  'away_score': 4,
  'winner': 'Away'},
 73997: {'home_team': 'Jays',
  'away_team': 'Orioles',
  'home_score': 3,
  'away_score': 7,
  'winner': 'Away'},
 74004: {'home_team': 'Braves',
  'away_team': 'Mets',
  'home_score': 13,
  'away_score': 1,
  'winner': 'Home'},
 74007: {'home_team': 'Twins',
  'away_team': 'Royals',
  'home_sco

In [4]:
odds_match_dict = get_mlb_odds(score_match_dict, date)
odds_match_dict

{73663: 'Home',
 74005: 'Away',
 73999: 'Away',
 74006: 'Away',
 74000: 'Home',
 74002: 'Home',
 74004: 'Home',
 74007: 'Home',
 73997: 'Home'}

In [5]:
odds_match_dict

{73663: 'Home',
 74005: 'Away',
 73999: 'Away',
 74006: 'Away',
 74000: 'Home',
 74002: 'Home',
 74004: 'Home',
 74007: 'Home',
 73997: 'Home'}

In [6]:
# get lists of correct and incorrect predictions
correct_ids = []
upset_ids = []

for match_id in odds_match_dict.keys():
    if odds_match_dict[match_id]!=score_match_dict[match_id]['winner']:
       upset_ids.append(match_id) 
    else:
        correct_ids.append(match_id)

def create_tweet(correct_ids, upset_ids, score_match_dict, date='2022-08-11'):
    tweet_text = str('#MLB Scores {date}\n\n').format(date=date)
    tweet_text=tweet_text+str("UPSETS: \n")
    for id in upset_ids:
        score_match_dict[id]
        tweet_text=tweet_text+str("{home_team}: {home_score} - {away_team}: {away_score} \n").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score'])

    tweet_text=tweet_text+str("\n EXPECTED: \n")
    for id in correct_ids:
        score_match_dict[id]
        tweet_text=tweet_text+(str("{home_team}: {home_score} - {away_team}: {away_score} \n").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score']))

    if len(tweet_text)>=280:
        tweet_text_list = tweet_text.split('EXPECTED')
        tweet_text_list[1]='EXPECTED'+tweet_text_list[1]
    else:
        tweet_text_list=[tweet_text]

    return tweet_text_list

tweet_text_list = create_tweet(correct_ids, upset_ids, score_match_dict, date=date)

In [7]:
for tweet_text in (tweet_text_list):
    print(tweet_text)

#MLB Scores 2022-08-15

UPSETS: 
Nationals: 5 - Cubs: 4 
Marlins: 3 - Padres: 0 
Yankees: 0 - Rays: 4 
Guardians: 5 - Tigers: 7 
Jays: 3 - Orioles: 7 

 EXPECTED: 
Guardians: 4 - Tigers: 1 
Reds: 3 - Phillies: 4 
Braves: 13 - Mets: 1 
Twins: 4 - Royals: 2 

